### 一个简单的线性回归示例

构造一组输入数据x和其对应的标签y

In [3]:
import numpy as np

In [4]:
x_values = [i for i in range(11)]
x_train = np.array(x_values, dtype=np.float32)
x_train = x_train.reshape(-1,1)
x_train.shape

(11, 1)

In [5]:
y_values = [2*i for i in range(11)]
y_train = np.array(y_values, dtype=np.float32)
y_train = y_train.reshape(-1,1)
y_train.shape

(11, 1)

In [6]:
import torch
import torch.nn as nn

### 线性回归模型
 - 线性回归：一个不加激活函数的全连接层

In [7]:
class LinearRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)
        
    def forward(self, x):
        out = self.linear(x)
        return out

In [9]:
input_dim=1
output_dim=1

model = LinearRegressionModel(input_dim, output_dim)
model

LinearRegressionModel(
  (linear): Linear(in_features=1, out_features=1, bias=True)
)

指定参数和损失函数

In [12]:
epochs = 1000
learning_ratr = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_ratr)
criterion = nn.MSELoss()

训练模型

In [14]:
for epoch in range(epochs):
    epoch += 1
    # 数据转为tensor格式
    inputs = torch.from_numpy(x_train)
    label = torch.from_numpy(y_train)
    # 梯度清零
    optimizer.zero_grad()
    # 前向传播
    outputs = model(inputs)
    # 计算损失
    loss = criterion(outputs, label)
    # 反向传播
    loss.backward()
    # 优化迭代更新参数
    optimizer.step()
    
    # 打印相关的信息
    if epoch % 50 == 0:
        print('epoch {},  loss {}'.format(epoch, loss.detach().item()))
    

epoch 50,  loss 1.8544662907515885e-06
epoch 100,  loss 1.057708232110599e-06
epoch 150,  loss 6.033063755239709e-07
epoch 200,  loss 3.4431349149599555e-07
epoch 250,  loss 1.9635518810900976e-07
epoch 300,  loss 1.1191932713927599e-07
epoch 350,  loss 6.383306327961691e-08
epoch 400,  loss 3.6413386084177546e-08
epoch 450,  loss 2.0803865297125412e-08
epoch 500,  loss 1.1852460346517546e-08
epoch 550,  loss 6.745045855183207e-09
epoch 600,  loss 3.868805276141529e-09
epoch 650,  loss 2.192278802937153e-09
epoch 700,  loss 1.2461554010911868e-09
epoch 750,  loss 7.193050044485005e-10
epoch 800,  loss 4.0872746853715114e-10
epoch 850,  loss 2.3131084547767244e-10
epoch 900,  loss 1.3421619371456472e-10
epoch 950,  loss 7.325604706176492e-11
epoch 1000,  loss 4.3637451291322904e-11


### 测试模型并预测结果

In [16]:
predicted = model(torch.from_numpy(x_train).requires_grad_()).data.numpy()
predicted

array([[1.2312010e-05],
       [2.0000105e+00],
       [4.0000086e+00],
       [6.0000072e+00],
       [8.0000048e+00],
       [1.0000004e+01],
       [1.2000002e+01],
       [1.4000000e+01],
       [1.5999998e+01],
       [1.7999996e+01],
       [1.9999994e+01]], dtype=float32)

### 模型的保存和读取

In [17]:
torch.save(model.state_dict(), 'model.pkl')
model.load_state_dict(torch.load('model.pkl'))

<All keys matched successfully>

### 使用GPU训练
- 只需要把**数据和模型**传入到cuda里面就可以了

In [20]:
import torch
import torch.nn as nn
import numpy as np


class LinearRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)
        
    def forward(self, x):
        out = self.linear(x)
        return out

input_dim=1
output_dim=1

model = LinearRegressionModel(input_dim, output_dim)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
print(device)
epochs = 1000
learning_ratr = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_ratr)
criterion = nn.MSELoss()

for epoch in range(epochs):
    epoch += 1
    # 数据转为tensor格式
    inputs = torch.from_numpy(x_train).to(device)
    label = torch.from_numpy(y_train).to(device)
    # 梯度清零
    optimizer.zero_grad()
    # 前向传播
    outputs = model(inputs)
    # 计算损失
    loss = criterion(outputs, label)
    # 反向传播
    loss.backward()
    # 优化迭代更新参数
    optimizer.step()
    
    # 打印相关的信息
    if epoch % 50 == 0:
        print('epoch {},  loss {}'.format(epoch, loss.detach().item()))

cuda:0
epoch 50,  loss 0.0012587368255481124
epoch 100,  loss 0.0007179318927228451
epoch 150,  loss 0.00040948358946479857
epoch 200,  loss 0.00023354956647381186
epoch 250,  loss 0.00013320926518645138
epoch 300,  loss 7.597674266435206e-05
epoch 350,  loss 4.333373726694845e-05
epoch 400,  loss 2.471643892931752e-05
epoch 450,  loss 1.4097093298914842e-05
epoch 500,  loss 8.040269676712342e-06
epoch 550,  loss 4.5863112063671e-06
epoch 600,  loss 2.615967559904675e-06
epoch 650,  loss 1.4921160982339643e-06
epoch 700,  loss 8.509024382874486e-07
epoch 750,  loss 4.852959136769641e-07
epoch 800,  loss 2.76942017762849e-07
epoch 850,  loss 1.5789576934821525e-07
epoch 900,  loss 9.000461176356112e-08
epoch 950,  loss 5.140690717553298e-08
epoch 1000,  loss 2.9284267455409463e-08
